# Sqlalcmemy 1.4/2.0 Tutorial

## Establishing Connectity - the Engine

In [1]:
from sqlalchemy import create_engine
engine = create_engine(
    "sqlite+pysqlite:///:memory:", 
    echo=True, future=True)

## Working with Transactions and the DBAPI

### Getting a Connection

In [2]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(
        text("select 'hello world'"))
    print(result.all())

2022-04-01 06:57:39,864 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 06:57:39,866 INFO sqlalchemy.engine.Engine select 'hello world'
2022-04-01 06:57:39,867 INFO sqlalchemy.engine.Engine [generated in 0.00314s] ()
[('hello world',)]
2022-04-01 06:57:39,871 INFO sqlalchemy.engine.Engine ROLLBACK


### Committing Changes

In [3]:
with engine.connect() as conn:
    conn.execute(
        text("CREATE TABLE "
        "some_table "
        "(x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table "
        "(x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, 
        {"x": 2, "y": 4}]
    )
    conn.commit()


2022-04-01 06:58:53,098 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 06:58:53,099 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2022-04-01 06:58:53,101 INFO sqlalchemy.engine.Engine [generated in 0.00309s] ()
2022-04-01 06:58:53,103 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-01 06:58:53,104 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ((1, 1), (2, 4))
2022-04-01 06:58:53,107 INFO sqlalchemy.engine.Engine COMMIT


The "begin once" style

In [4]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO"
        " some_table (x, y)"
        " VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, 
        {"x": 9, "y": 10}]
    )

2022-04-01 07:00:38,595 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 07:00:38,596 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-01 07:00:38,598 INFO sqlalchemy.engine.Engine [cached since 105.5s ago] ((6, 8), (9, 10))
2022-04-01 07:00:38,602 INFO sqlalchemy.engine.Engine COMMIT


### Basics of Statement Execution

#### Fetching Rows

In [5]:
with engine.connect() as conn:
    result = conn.execute(
        text(
            "SELECT x, y "
            "FROM some_table"))
    for row in result:
        print(f"x: {row.x} "
        f" y: {row.y}")

2022-04-01 07:02:28,173 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 07:02:28,174 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2022-04-01 07:02:28,175 INFO sqlalchemy.engine.Engine [generated in 0.00181s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-04-01 07:02:28,176 INFO sqlalchemy.engine.Engine ROLLBACK


#### Sending Parameters

In [6]:
with engine.connect() as conn:
    result = conn.execute(
        text("SELECT x, y FROM some_table WHERE y > :y"),
        {"y": 2}
    )
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-01 07:03:27,509 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 07:03:27,511 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2022-04-01 07:03:27,512 INFO sqlalchemy.engine.Engine [generated in 0.00235s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-04-01 07:03:27,514 INFO sqlalchemy.engine.Engine ROLLBACK


#### Sending Multiple Parameters

In [7]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}]
    )
    conn.commit()


2022-04-01 07:09:37,166 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 07:09:37,168 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-01 07:09:37,170 INFO sqlalchemy.engine.Engine [cached since 644.1s ago] ((11, 12), (13, 14))
2022-04-01 07:09:37,172 INFO sqlalchemy.engine.Engine COMMIT


#### Bunding Parameters with a Statement

In [8]:
stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-01 07:10:32,127 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 07:10:32,128 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-04-01 07:10:32,129 INFO sqlalchemy.engine.Engine [generated in 0.00255s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-04-01 07:10:32,142 INFO sqlalchemy.engine.Engine ROLLBACK


#### Executing with an ORM Session

In [9]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-01 07:11:08,508 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 07:11:08,509 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-04-01 07:11:08,511 INFO sqlalchemy.engine.Engine [cached since 36.38s ago] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-04-01 07:11:08,512 INFO sqlalchemy.engine.Engine ROLLBACK


commit as you go

In [10]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y":11}, {"x": 13, "y": 15}]
    )
    session.commit()


2022-04-01 07:11:42,195 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 07:11:42,197 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2022-04-01 07:11:42,198 INFO sqlalchemy.engine.Engine [generated in 0.00129s] ((11, 9), (15, 13))
2022-04-01 07:11:42,200 INFO sqlalchemy.engine.Engine COMMIT


## Working with Database Metadata

### Setting up MetaData with Table objects

In [11]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [12]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)

Column

In [13]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [14]:
user_table.c.keys()

['id', 'name', 'fullname']

### Declaring Simple Constraints

In [15]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

In [17]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column('id', Integer, 
    primary_key=True),
    Column('user_id',
     ForeignKey('user_account.id'), 
     nullable=False),
    Column('email_address',
     String,
     nullable=False)
)

### Emitting DDL to the Database

In [19]:
metadata_obj.create_all(engine)

2022-04-01 08:12:10,898 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 08:12:10,898 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-01 08:12:10,900 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-01 08:12:10,902 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-01 08:12:10,903 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-01 08:12:10,906 INFO sqlalchemy.engine.Engine COMMIT


### Defining Table Metadata with the ORM

#### Setting up the Registry

In [20]:
from sqlalchemy.orm import registry
mapper_registry = registry()

In [21]:
mapper_registry.metadata

MetaData()

In [22]:
Base = mapper_registry.\
    generate_base()

#### Declaring Mapped Classes

In [23]:
from sqlalchemy.orm import relationship
class User(Base):
    __tablename__ = 'user_account'
    id = Column(Integer, 
    primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address",
     back_populates="user")
    def __repr__(self):
       return f"User(id={self.id!r}," \
       f" name={self.name!r}," \
       f" fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, 
    primary_key=True)
    email_address = Column(String,
     nullable=False)
    user_id = Column(Integer, 
    ForeignKey('user_account.id'))
    user = relationship("User", 
    back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r},"\
        f" email_address="\
        f"{self.email_address!r})"

In [24]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

#### Other Mapped Class Details

the classes have an automatically generated `__init__()` method

In [26]:
sandy = User(name='sandy',
 fullname='Sandy Cheeks')

we provided a `__repr__()` method

In [27]:
sandy

User(id=None, name='sandy', fullname='Sandy Cheeks')

we also included a bidirectional relationship

#### Emitting DDL to the database

In [28]:
# emit CREATE statements given ORM registry
mapper_registry.metadata.create_all(engine)

# the identical MetaData object is also present on the
# declarative base
Base.metadata.create_all(engine)

2022-04-01 10:37:25,834 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 10:37:25,838 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-01 10:37:25,839 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-01 10:37:25,841 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-01 10:37:25,842 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-01 10:37:25,844 INFO sqlalchemy.engine.Engine COMMIT
2022-04-01 10:37:25,845 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 10:37:25,846 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-01 10:37:25,847 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-01 10:37:25,848 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-01 10:37:25,849 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-01 10:37:25,850 INFO sqlalchemy.engine.Engine COMMIT


#### Combining Core Table Declarations with ORM Declarative

In [29]:
class User(Base):
    __table__ = user_table

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User({self.name!r}, {self.fullname!r})"

class Address(Base):
    __table__ = address_table

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address({self.email_address!r})"

C:\Users\18463\AppData\Local\Temp\ipykernel_23008\330721587.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  class User(Base):
C:\Users\18463\AppData\Local\Temp\ipykernel_23008\330721587.py:9: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Address, and will be replaced in the string-lookup table.
  class Address(Base):


### Table Reflection

In [30]:
some_table = Table("some_table",
 metadata_obj, autoload_with=engine)


2022-04-01 11:27:59,451 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-01 11:27:59,453 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-04-01 11:27:59,454 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-01 11:27:59,456 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-01 11:27:59,456 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-04-01 11:27:59,470 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2022-04-01 11:27:59,472 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-01 11:27:59,475 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2022-04-01 11:27:59,475 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-01 11:27:59,477 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
20

In [31]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)